## FFORMA results using R data 

In [57]:
import pandas as pd

from ESRNN.m4_data import prepare_m4_data, seas_dict
from ESRNN.utils_evaluation import evaluate_prediction_owa

from fforma import FFORMA

from functools import partial
import multiprocessing as mp
import glob
from tsfeatures import tsfeatures
freqs = {'Hourly': 24, 'Daily': 1,
         'Monthly': 12, 'Quarterly': 4,
         'Weekly':1, 'Yearly': 1}
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Getting errors and features for the holdout set.

In [58]:
train_errors = pd.read_csv('R/data/train-errors-fforma.csv').set_index('unique_id').sort_index()
train_feats = pd.read_csv('R/data/train-feats-fforma.csv').set_index('unique_id').sort_index()

train_feats.head()

,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,ARCH.LM,crossing_points,entropy,...,seasonal_strength,peak,trough,stability,hw_alpha,hw_beta,hw_gamma,unitroot_kpss,unitroot_pp,series_length
unique_id,,,,,,,,,,,,,,,,,,,,,
D1,0.997054,9.683578,-0.025463,0.007460,-0.517817,0.271951,0.0,0.997038,3,0.326438,...,0.0,0,0,1.002848,0.0,0.0,0.0,12.257000,0.771839,992
D10,0.985281,8.412466,0.001281,0.019863,-0.521245,0.295101,0.0,0.943606,59,0.508607,...,0.0,0,0,0.972241,0.0,0.0,0.0,4.452811,-7.498107,660
D100,0.994504,9.403108,0.001704,0.001222,-0.503351,0.255230,0.0,0.985382,12,0.404790,...,0.0,0,0,0.999791,0.0,0.0,0.0,7.098207,-4.807179,992
D1000,0.989473,9.003030,-0.121437,0.022924,-0.549774,0.321185,0.0,0.987966,31,0.435082,...,0.0,0,0,0.953607,0.0,0.0,0.0,9.158440,-3.319396,1038
D1001,0.982148,8.475112,-0.134744,0.026297,-0.560199,0.329448,0.0,0.976176,57,0.531104,...,0.0,0,0,0.939021,0.0,0.0,0.0,7.101309,-8.726363,1038


In [59]:
train_errors.head()

,auto_arima_forec,ets_forec,nnetar_forec,tbats_forec,stlm_ar_forec,rw_drift_forec,theta_forec,naive_forec,snaive_forec
unique_id,,,,,,,,,
D1,0.353925,0.357937,1.471586,0.325105,26.459177,0.353925,0.436777,0.583062,0.583062
D10,0.657762,0.608243,1.252326,0.736966,0.843755,0.550240,0.580952,0.608266,0.608266
D100,0.722749,0.722680,1.643707,0.723167,9.400545,0.839411,0.851346,0.722749,0.722749
D1000,0.388785,0.271660,0.214388,0.234226,7.543362,0.344468,0.305202,0.242098,0.242098
D1001,0.468595,0.487338,0.153999,0.216409,5.574069,0.542113,0.510042,0.450602,0.450602


In [60]:
train_feats.head()

,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,ARCH.LM,crossing_points,entropy,...,seasonal_strength,peak,trough,stability,hw_alpha,hw_beta,hw_gamma,unitroot_kpss,unitroot_pp,series_length
unique_id,,,,,,,,,,,,,,,,,,,,,
D1,0.997054,9.683578,-0.025463,0.007460,-0.517817,0.271951,0.0,0.997038,3,0.326438,...,0.0,0,0,1.002848,0.0,0.0,0.0,12.257000,0.771839,992
D10,0.985281,8.412466,0.001281,0.019863,-0.521245,0.295101,0.0,0.943606,59,0.508607,...,0.0,0,0,0.972241,0.0,0.0,0.0,4.452811,-7.498107,660
D100,0.994504,9.403108,0.001704,0.001222,-0.503351,0.255230,0.0,0.985382,12,0.404790,...,0.0,0,0,0.999791,0.0,0.0,0.0,7.098207,-4.807179,992
D1000,0.989473,9.003030,-0.121437,0.022924,-0.549774,0.321185,0.0,0.987966,31,0.435082,...,0.0,0,0,0.953607,0.0,0.0,0.0,9.158440,-3.319396,1038
D1001,0.982148,8.475112,-0.134744,0.026297,-0.560199,0.329448,0.0,0.976176,57,0.531104,...,0.0,0,0,0.939021,0.0,0.0,0.0,7.101309,-8.726363,1038


In [61]:
import pandas as pd
import matplotlib.pyplot as plt


gen_1=pd.read_csv('data/Kaggle_solar/Plant_1_Generation_Data.csv')
gen_1.drop('PLANT_ID', axis=1)
sens_1= pd.read_csv('data/Kaggle_solar/Plant_1_Weather_Sensor_Data.csv')
sens_1.drop('PLANT_ID',axis=1)
#format datetime
gen_1['DATE_TIME']= pd.to_datetime(gen_1['DATE_TIME'],format='%d-%m-%Y %H:%M')
sens_1['DATE_TIME']= pd.to_datetime(sens_1['DATE_TIME'],format='%Y-%m-%d %H:%M:%S')

In [62]:
univariate_df = gen_1.copy()
univariate_df = univariate_df.groupby('DATE_TIME').sum()
univariate_df = univariate_df['DAILY_YIELD'].reset_index()
univariate_df.rename(columns={'DAILY_YIELD': 'y', 'DATE_TIME': 'ds'}, inplace=True)
univariate_df['date_time'] = univariate_df['ds']
univariate_df.set_index('date_time', inplace=True)
univariate_df['unique_id'] = 1
univariate_df.head()

,ds,y,unique_id
date_time,,,
2020-05-15 00:00:00,2020-05-15 00:00:00,0.0,1
2020-05-15 00:15:00,2020-05-15 00:15:00,0.0,1
2020-05-15 00:30:00,2020-05-15 00:30:00,0.0,1
2020-05-15 00:45:00,2020-05-15 00:45:00,0.0,1
2020-05-15 01:00:00,2020-05-15 01:00:00,0.0,1


Getting features and predictions for the whole set.

In [63]:
feats = pd.read_csv('R/data/pred-feats-fforma.csv').set_index('unique_id').sort_index()
preds = [pd.read_csv(file) for file in glob.glob('R/data/preds-fforma-*.csv')]
preds = pd.concat(preds).set_index(['unique_id', 'ds']).sort_index()

In [64]:
feats.head()

,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,ARCH.LM,crossing_points,entropy,...,seasonal_strength,peak,trough,stability,hw_alpha,hw_beta,hw_gamma,unitroot_kpss,unitroot_pp,series_length
unique_id,,,,,,,,,,,,,,,,,,,,,
D1,0.997059,9.687135,-0.029772,0.007627,-0.518825,0.273454,0.0,0.997072,9,0.327261,...,0.0,0,0,0.992386,0.0,0.0,0.0,12.443481,0.812605,1006
D10,0.984991,8.366800,0.002487,0.020569,-0.517569,0.293474,0.0,0.948765,53,0.508987,...,0.0,0,0,0.953484,0.0,0.0,0.0,4.749779,-6.458571,674
D100,0.994623,9.415191,0.001736,0.001220,-0.503348,0.255226,0.0,0.984972,10,0.412695,...,0.0,0,0,1.001483,0.0,0.0,0.0,7.236602,-4.904598,1006
D1000,0.990309,9.095817,-0.118990,0.022970,-0.549107,0.320950,0.0,0.988706,33,0.428637,...,0.0,0,0,0.988355,0.0,0.0,0.0,9.388402,-3.344469,1052
D1001,0.983824,8.616251,-0.133697,0.026031,-0.560066,0.329184,0.0,0.978129,65,0.518474,...,0.0,0,0,0.975697,0.0,0.0,0.0,7.370831,-8.782178,1052


In [65]:
preds.head()

auto_arima_forec    ets_forec  nnetar_forec  tbats_forec  \
unique_id ds                                                             
D1        1        2030.707562  2030.782164   2026.136678  2031.007655   
          2        2031.715124  2031.784536   2022.776206  2032.235579   
          3        2032.722687  2032.786908   2019.604045  2033.463504   
          4        2033.730249  2033.789280   2016.606992  2034.691429   
          5        2034.737811  2034.791652   2013.773026  2035.919354   

              stlm_ar_forec  rw_drift_forec  theta_forec  naive_forec  \
unique_id ds                                                            
D1        1     1972.806338     2030.707562  2030.257390       2029.7   
          2     1853.490129     2031.715124  2030.792255       2029.7   
          3     1688.666136     2032.722687  2031.327120       2029.7   
          4     1532.134587     2033.730249  2031.861986       2029.7   
          5     1450.650226     2034.737811  2032.396851       2029.7   

              snaive_forec  
unique_id ds                
D1        1         2029.7  
          2         2029.7  
          3         2029.7  
          4         2029.7  
          5         2029.7

Instantiating and training the model. 

In [66]:
optimal_params = {'n_estimators': 94,
                  'eta': 0.58,
                  'max_depth': 14,
                  'subsample': 0.92,
                  'colsample_bytree': 0.77}
model = FFORMA(params=optimal_params, verbose_eval=20)

Fit model

In [67]:
model.fit(errors=train_errors, holdout_feats=train_feats, feats=feats)


Training until validation scores don't improve for 10 rounds
[20]	training's FFORMA-loss: 0.760744	valid_1's FFORMA-loss: 0.771149
[40]	training's FFORMA-loss: 0.7362	valid_1's FFORMA-loss: 0.752686
[60]	training's FFORMA-loss: 0.724347	valid_1's FFORMA-loss: 0.745992
[80]	training's FFORMA-loss: 0.716198	valid_1's FFORMA-loss: 0.742493
Did not meet early stopping. Best iteration is:
[94]	training's FFORMA-loss: 0.71184	valid_1's FFORMA-loss: 0.741106


Making predictions

In [68]:

from tsfeatures import tsfeatures
hourly_df = univariate_df.resample('H').mean()
solar_feats = tsfeatures(hourly_df,freq=24)
solar_feats = solar_feats.dropna(axis=1)
model.new_weights(solar_feats)

array([[7.75128701e-01, 8.95129588e-02, 1.74339495e-06, 7.75434675e-02,
        1.53244770e-07, 5.69903782e-02, 7.51101068e-04, 2.12505626e-05,
        5.02467020e-05]])

In [71]:
from sklearn.ensemble import VotingRegressor
from fforma.r_models import ARIMA, ETS, NNETAR
# Initialize models
arima_model = ARIMA(freq=24)


OSError: cannot load library '/usr/lib64/R/lib/libR.so': /home/jhivandb/miniconda3/envs/fforma/lib/python3.6/site-packages/zmq/backend/cython/../../../../.././libstdc++.so.6: version `GLIBCXX_3.4.30' not found (required by /usr/lib/libicuuc.so.74)

In [41]:
fforma_preds = model.predict(preds).reset_index()

           auto_arima_forec  ets_forec  nnetar_forec  tbats_forec  \
unique_id                                                           
D1                 0.391758   0.206299      0.000016     0.206298   
D10                0.137145   0.105485      0.001432     0.204554   
D100               0.009008   0.208581      0.000222     0.226841   
D1000              0.112626   0.134271      0.000902     0.186176   
D1001              0.147013   0.140808      0.000861     0.195028   
...                     ...        ...           ...          ...   
Y9995              0.089272   0.021174      0.055955     0.046550   
Y9996              0.113181   0.032869      0.000029     0.153438   
Y9997              0.097820   0.053370      0.000096     0.043016   
Y9998              0.315214   0.258421      0.000029     0.382358   
Y9999              0.424861   0.251698      0.000163     0.289601   

           stlm_ar_forec  rw_drift_forec  theta_forec  naive_forec  \
unique_id                       

In [42]:
fforma_preds.head()

,unique_id,ds,auto_arima_forec,ets_forec,nnetar_forec,tbats_forec,stlm_ar_forec,rw_drift_forec,theta_forec,naive_forec,snaive_forec,fforma_prediction
0,D1,1,2030.707562,2030.782164,2026.136678,2031.007655,1972.806338,2030.707562,2030.257390,2029.7,2029.7,2030.709395
1,D1,2,2031.715124,2031.784536,2022.776206,2032.235579,1853.490129,2031.715124,2030.792255,2029.7,2029.7,2031.683294
2,D1,3,2032.722687,2032.786908,2019.604045,2033.463504,1688.666136,2032.722687,2031.327120,2029.7,2029.7,2032.657196
3,D1,4,2033.730249,2033.789280,2016.606992,2034.691429,1532.134587,2033.730249,2031.861986,2029.7,2029.7,2033.631101
4,D1,5,2034.737811,2034.791652,2013.773026,2035.919354,1450.650226,2034.737811,2032.396851,2029.7,2029.7,2034.605009


Postprocessing

In [43]:
fforma_preds.loc[fforma_preds['fforma_prediction']<0, 'fforma_prediction'] = 0

fforma_preds

,unique_id,ds,auto_arima_forec,ets_forec,nnetar_forec,tbats_forec,stlm_ar_forec,rw_drift_forec,theta_forec,naive_forec,snaive_forec,fforma_prediction
0,D1,1,2030.707562,2030.782164,2026.136678,2031.007655,1972.806338,2030.707562,2030.257390,2029.7,2029.7,2030.709395
1,D1,2,2031.715124,2031.784536,2022.776206,2032.235579,1853.490129,2031.715124,2030.792255,2029.7,2029.7,2031.683294
2,D1,3,2032.722687,2032.786908,2019.604045,2033.463504,1688.666136,2032.722687,2031.327120,2029.7,2029.7,2032.657196
3,D1,4,2033.730249,2033.789280,2016.606992,2034.691429,1532.134587,2033.730249,2031.861986,2029.7,2029.7,2033.631101
4,D1,5,2034.737811,2034.791652,2013.773026,2035.919354,1450.650226,2034.737811,2032.396851,2029.7,2029.7,2034.605009
...,...,...,...,...,...,...,...,...,...,...,...,...
1277712,Y9999,2,10635.706312,10791.286919,9475.368859,11879.481575,3955.125000,10269.608696,9905.601312,9517.0,9517.0,11021.969996
1277713,Y9999,3,11195.059468,11428.467514,9467.076841,13045.200197,3955.125000,10645.913043,10099.938051,9517.0,9517.0,11769.971795
1277714,Y9999,4,11754.412623,12065.648109,9462.441083,14324.895503,3955.125000,11022.217391,10294.274791,9517.0,9517.0,12550.981940
1277715,Y9999,5,12313.765779,12702.828705,9459.846051,15729.670987,3955.125000,11398.521739,10488.611530,9517.0,9517.0,13368.215750


Evaluating results

In [ ]:
def evaluate_fforma(dataset_name, fforma_df, directory, num_obs):
    print(dataset_name)
    _, y_train_df, X_test_df, y_test_df = prepare_m4_data(dataset_name=dataset_name,
                                                          directory=directory,
                                                          num_obs=num_obs)
    
    y_test_fforma = fforma_preds[fforma_df['unique_id'].isin(y_test_df['unique_id'].unique())]
    y_test_fforma = y_test_fforma.rename(columns={'fforma_prediction': 'y_hat'})
    y_test_fforma = y_test_fforma.filter(items=['unique_id', 'ds', 'y_hat'])
    
    seasonality = freqs[dataset_name]
    owa, mase, smape = evaluate_prediction_owa(y_test_fforma, y_train_df, X_test_df, y_test_df, seasonality)
    
    return dataset_name, owa, mase, smape

In [ ]:
evaluate_fforma_p = partial(evaluate_fforma, fforma_df=fforma_preds, directory='./data', num_obs=100000)

In [ ]:
with mp.Pool() as pool:
    eval_fforma = pool.map(evaluate_fforma_p, freqs.keys())

Hourly
Daily
Monthly
Quarterly
Successfully downloaded M4-info.csv 4335598 bytes.
Weekly
Yearly
Successfully downloaded M4-info.csv 4335598 bytes.
Successfully downloaded M4-info.csv 4335598 bytes.
Successfully downloaded Train/Daily-train.csv 95765153 bytes.
Successfully downloaded Train/Hourly-train.csv 2347115 bytes.
Successfully downloaded Train/Daily-train.csv 95765153 bytes.
Successfully downloaded Train/Daily-train.csv 95765153 bytes.
Successfully downloaded Train/Daily-train.csv 95765153 bytes.
Successfully downloaded Train/Weekly-train.csv 4015067 bytes.
Successfully downloaded Train/Yearly-train.csv 25355736 bytes.
Successfully downloaded Test/Daily-test.csv 576459 bytes.
Successfully downloaded Test/Hourly-test.csv 132820 bytes.
Successfully downloaded Test/Monthly-test.csv 7942698 bytes.
Successfully downloaded Test/Quarterly-test.csv 1971754 bytes.
Successfully downloaded Test/Weekly-test.csv 44247 bytes.
Successfully downloaded Test/Yearly-test.csv 1486434 bytes.


Succes

KeyboardInterrupt: 

In [ ]:
eval_fforma